In [1]:
import random
from tqdm import tqdm
import pandas as pd

In [2]:
dict_of_value = {
    "flower": {
        "hp": 4780,
    },
    "plume":{
        "atk": 311,
    },
    "sands": {
        "hp%": 46.6, "atk%": 46.6, "def%": 58.3,
        "er": 51.8, "em": 186.5,      
    },
    "goblet": {
        "hp%": 46.6, "atk%": 46.6, "def%": 58.3,
        "pyro": 46.6, "electro": 46.6, "cryo": 46.6, "hydro": 46.6,
        "dendro": 46.6, "anemo": 46.6, "geo": 46.6, "physical": 58.3,
        "em": 186.5 
    },
    "circlet": {
        "hp%": 46.6, "atk%": 46.6, "def%": 58.3,
        "cr": 31.1, "cd": 62.2, "heal": 35.9,
        "em": 186.5,
    },
    "sub": {
        "hp": 298.75, "atk": 19.45, "def":23.15,
        "hp%": 5.83, "atk%": 5.83, "def%": 7.29, "er": 6.48, "em": 23.31,
        "cr": 3.89, "cd": 7.77,  
    }
}

In [3]:
dict_of_prob = {
    "flower": {
        "hp": 1,
    },
    "plume":{
        "atk": 1,
    },
    "sands": {
        "hp%": 0.8 / 3, "atk%": 0.8 / 3, "def%": 0.8 / 3,
        "er": 0.1, "em": 0.1,      
    },
    "goblet": {
        "hp%": 0.575 / 3, "atk%": 0.575 / 3, "def%": 0.575 / 3,
        "pyro": 0.05, "electro": 0.05, "cryo": 0.05, "hydro": 0.05,
        "dendro": 0.05, "anemo": 0.05, "geo": 0.05, "physical": 0.05,
        "em": 0.025 
    },
    "circlet": {
        "hp%": 0.22, "atk%": 0.22, "def%": 0.22,
        "cr": 0.1, "cd": 0.1, "heal": 0.1,
        "em": 0.04,
    },
    "sub": {
        "hp": 0.15, "atk": 0.15, "def": 0.15,
        "hp%": 0.1, "atk%": 0.1, "def%": 0.1, "er": 0.1, "em": 0.1,
        "cr": 0.075, "cd": 0.075,  
    }
}

In [4]:
def get_base(the_position):
    
    # sim main
    
    the_pool = dict_of_prob[the_position].copy()    
    the_choice = random.choices(population = list(the_pool.keys()), weights = list(the_pool.values()), k = 1)
    the_main_name = the_choice[0]
    
        
    # sim sub
    
    the_pool = dict_of_prob["sub"].copy()
    try: del the_pool[the_main_name]
    except Exception: pass
    
    the_sub_names = []
    for i in range(3 + random.randint(0, 1)):
        the_choice = random.choices(population = list(the_pool.keys()), weights = list(the_pool.values()), k = 1)
        the_sub_names.append(the_choice[0])
        del the_pool[the_choice[0]]
        
    the_output = {
        the_position: the_main_name,
        "sub": the_sub_names
    }
    
    return the_output

In [5]:
# for i in range(20):
#     print(get_base("goblet"))

In [6]:
def get_upgrade(the_position, the_base):
    
    the_pool = dict_of_prob["sub"].copy()
    try: del the_pool[the_base["the_position"]]
    except Exception: pass
    for k in the_base["sub"]:
        del the_pool[k]
    
    
    the_upgrade_names = []
    num_upgrade = 5
    
    if len(the_base["sub"]) == 3:        
        the_choice = random.choices(population = list(the_pool.keys()), weights = list(the_pool.values()), k = 1)
        the_upgrade_names.append(the_choice[0])
        num_upgrade = num_upgrade - 1
        

    the_choice = random.choices(
        population = the_base["sub"] + the_upgrade_names, 
        weights = [1, 1, 1, 1],
        k = num_upgrade)
    
    the_upgrade_names.extend(the_choice)
    
    
    the_output = {
        the_position: the_base[the_position],
        "sub": the_base["sub"] + the_upgrade_names
    }
    
    return the_output

In [7]:
# for i in range(20):
#     the_base = get_base("goblet")
#     print(get_upgrade("goblet", the_base))

In [8]:
def get_value(the_position, the_dict):
    
    
    the_main_name = the_dict[the_position]
    the_main_value = {
        the_main_name: dict_of_value[the_position][the_main_name]
    }
    
    the_sub_value = {}
    
    for the_sub_name in the_dict["sub"]:
        
        if the_sub_name not in the_sub_value:
            the_sub_value[the_sub_name] = 0
        
        the_choice = random.choices(
            population = [ i / 10 * dict_of_value["sub"][the_sub_name] for i in range(7,11)], 
            weights = [1, 1, 1, 1],
            k = 1)
        
        the_sub_value[the_sub_name] += the_choice[0]
        
    the_output = {
        "position": the_position,
        "main": the_main_value,
        "sub": the_sub_value
    }
     
    return the_output

In [9]:
# for i in range(20):
#     the_base = get_base("goblet")
#     the_dict = get_upgrade("goblet", the_base)
#     print(get_value("goblet", the_dict))

In [10]:
def artifact_sim():
    
    the_choice = random.choices(
        population = ["flower", "plume", "sands", "goblet", "circlet"], 
        weights = [1, 1, 1, 1, 1],
        k = 1)
    the_position = the_choice[0]
    
    the_base = get_base(the_position)
    the_dict = get_upgrade(the_position, the_base)
    the_value = get_value(the_position, the_dict)
    
    # convert to dataframe row
    
    the_output = {
        "position": the_position,
        "main": the_base[the_position],
    }
    
    the_output.update(the_value["sub"])
    
    return the_output

In [11]:
the_result = []

for i in tqdm(range(10000000)):
    
    tmp = artifact_sim()
    the_result.append(tmp)

100%|██████████| 10000000/10000000 [07:11<00:00, 23191.29it/s]


In [12]:
df = pd.DataFrame(the_result)
df = df.fillna(0)

In [13]:
df

,position,main,hp%,atk%,def,def%,hp,atk,er,cr,cd,em
0,goblet,anemo,5.247,20.405,39.355,7.290,0.000,0.000,0.000,0.000,0.000,0.000
1,goblet,hp%,0.000,0.000,20.835,13.122,448.125,46.680,0.000,0.000,0.000,0.000
2,sands,atk%,0.000,0.000,39.355,0.000,239.000,35.010,15.552,0.000,0.000,0.000
3,circlet,atk%,9.328,0.000,0.000,0.000,0.000,56.405,5.832,6.613,0.000,0.000
4,flower,hp,0.000,13.409,0.000,0.000,0.000,48.625,0.000,0.000,5.439,37.296
...,...,...,...,...,...,...,...,...,...,...,...,...
9999995,goblet,pyro,0.000,5.247,53.245,0.000,298.750,0.000,0.000,0.000,0.000,81.585
9999996,goblet,atk%,0.000,0.000,83.340,0.000,507.875,33.065,4.536,0.000,0.000,0.000
9999997,flower,hp,0.000,0.000,16.205,13.122,0.000,31.120,0.000,0.000,20.202,0.000
9999998,goblet,def%,0.000,4.664,76.395,0.000,448.125,0.000,0.000,0.000,0.000,37.296


In [14]:
df.to_csv('artifact_database_full.csv', index=False)